In [ ]:
from bs4 import BeautifulSoup
from urllib.request import Request, urlopen
from urllib.error import HTTPError
import json
import re
total_count = 0
count = 0
main_url = "https://ddr.densho.org/narrators/?page={}"
for page_num in range(1, 38):

    link1 = Request(main_url.format(page_num), headers={'User-Agent': 'Mozilla/5.0'})
    url1 = urlopen(link1)

    data1 = url1.read()
    data1_soup = BeautifulSoup(data1)

    print("*** Page {} ***".format(page_num))
    for narrator_link in data1_soup.find_all('h4'):
        #print(narrator_link.a.get('href'))
        link2 = Request(narrator_link.a.get('href'), headers={'User-Agent': 'Mozilla/5.0'})
        #print(narrator_link.a.get('href'))
        url2 = urlopen(link2)

        data2 = url2.read()
        data2_soup = BeautifulSoup(data2)
        count = count + 1
        
        #print("*** Interview links page***")
        narrator = data2_soup.find_all("div", attrs={'class':'col-sm-8 col-md-8'})[0]
        if data2_soup.find_all("div", attrs={'class':'url'}) != []:
            interview_count = 1
            for interviews in data2_soup.find_all("div", attrs={'class':'url'}):
                #interview_count = 1
                result = {}
                result['Narrator_Name'] = ""
                result['Bio'] = ""
                result['Title'] = ""
                result['Interviewer'] = ""
                result['Location'] = ""
                result['Date'] = ""
                result['Densho ID'] = ""
                result['transcript'] = {}
                #print(interviews.a.get('href'))
                #print("Interview")
                #print(interviews.a.get('href'))
                # Narrator name and Bio...
                try:
                    result['Narrator_Name'] = narrator.h1.text.strip().replace('"', "")#re.sub('"', '\"', narrator.h1.text.strip())
                except:
                    result['Narrator_Name'] = ""
                    
                try:
                    result['Bio'] = narrator.p.text
                except :
                    result['Bio'] = ""
                
                link3 = Request(interviews.a.get('href'), headers={'User-Agent': 'Mozilla/5.0'})
                #print(interviews.a.get('href'))
                url3 = urlopen(link3)

                data3 = url3.read()
                data3_soup = BeautifulSoup(data3)
                #count = 1
                #print("*** Segment Descriptions ***")
                
                num = 1
                for desc in data3_soup.find_all('tbody')[0].find_all('td'):
                    result['transcript']['segment {}'.format(num)] = {}
                    result['transcript']['segment {}'.format(num)]['Segment Desc'] = desc.text.split(' (', 1)[0].strip()
                    result['transcript']['segment {}'.format(num)]['Segment Questions'] = []
                    num = num + 1
                    
                for transcript_link in data3_soup.find_all('div', attrs={'id':'dls_tab'}):
                    print("*** Transcript link ***")
                    try:
                        print(transcript_link.find_all('a')[1].get('href'))
                        link4 = Request(transcript_link.find_all('a')[1].get('href'), headers={'User-Agent': 'Mozilla/5.0'})
                        url4 = urlopen(link4)

                        data4 = url4.read()
                        data4_soup = BeautifulSoup(data4)

                        #print("-----------------------------------------")
                        for data in data4_soup.find_all('div', attrs={'class':'segmentHead'}):
                            # title, interviewer, location, date, densho id...
                            for attr in data.text.splitlines():
                                if 'Title' in attr:
                                    #print(i.strip())
                                    result['Title'] = ' '.join(attr.strip().split()[1:])
                                if 'Interviewer' in attr:
                                    #print(i.strip())
                                    result['Interviewer'] = ' '.join(attr.strip().split()[1:])
                                if 'Location' in attr:
                                    #print(i.strip())
                                    result['Location'] = ' '.join(attr.strip().split()[1:])
                                if 'Date' in attr:
                                    #print(i.strip())
                                    result['Date'] = ' '.join(attr.strip().split()[1:])
                                if 'Densho ID' in attr:
                                    #print(i.strip())
                                    result['Densho ID'] = ' '.join(attr.strip().split()[2:])
                        
                        num = 1

                        for seg in data4_soup.find_all('div', attrs={'class':'segmentBody'}):
                            #print("*** Segment Body ***")
                            #result['transcript']['segment {}'.format(num)]['Segment Desc'] = ""
                            #result['transcript']['segment {}'.format(num)]['Segment Questions'] = re.sub("   +", "", "\n".join(p.text.strip() for p in seg.find_all('p')[1:-1]))
                            for p in seg.find_all('p')[1:-1]:
                                if '?' in p.text:
                                    #print(re.sub("  +", "",p.text))
                                    result['transcript']['segment {}'.format(num)]['Segment Questions'].append(re.sub('\s+',' ', re.sub("  +", "",p.text)))

                            '''
                            for p in seg.find_all('p')[1:-1]:
                                print(" ".join(p.text.strip().split()))
                            '''   
                            num = num + 1
                     
                    #except HTTPError as err:
                    except Exception as e:
                        print("No Transcripts")
                        print(e)
                        print(len(data3_soup.find_all('tbody')[0].find_all('td')))
                        result['Title'] = ""
                        result['Interviewer'] = ""
                        result['Location'] = ""
                        result['Date'] = ""
                        result['Densho ID'] = ""
                        #num = 1
                        result['transcript'] = {}
                        for num in range(0, len(data3_soup.find_all('tbody')[0].find_all('td'))):
                            result['transcript']['segment {}'.format(num+1)] = {}
                            result['transcript']['segment {}'.format(num+1)]['Segment Desc'] = ""
                            result['transcript']['segment {}'.format(num+1)]['Segment Questions'] = []
                            #num = num + 1
                                                   
                        

                #print()
                print(result)
                import os.path
                if os.path.isfile(r'C:\\Users\\maganti\\Documents\\INFO 5502 Chen Sir Project UNT\\Web Scraping Interviews\\Narrators Json Files-Updatedwithquestions\\{}.json'.format(result['Narrator_Name'] + '-' + str(interview_count))):
                    with open(r'C:\\Users\\maganti\\Documents\\INFO 5502 Chen Sir Project UNT\\Web Scraping Interviews\\Narrators Json Files-Updatedwithquestions\\{}.json'.format(result['Narrator_Name'] + '(1)' + '-' + str(interview_count)), 'w', encoding='utf-8') as f:
                        json.dump(result, f, ensure_ascii=False, indent=4)
                else:
                    with open(r'C:\\Users\\maganti\\Documents\\INFO 5502 Chen Sir Project UNT\\Web Scraping Interviews\\Narrators Json Files-Updatedwithquestions\\{}.json'.format(result['Narrator_Name'] + '-' + str(interview_count)), 'w', encoding='utf-8') as f:
                        json.dump(result, f, ensure_ascii=False, indent=4)
                print('***************************')
                interview_count = interview_count + 1 
             
        else:
            '''
            print("%%%%%%%%%%%%%%%%%%%%%%%%%")
            print("No Interviews....")
            print("%%%%%%%%%%%%%%%%%%%%%%%%%")
            '''
            result= {}
            result['Narrator_Name'] = narrator.h1.text.strip().replace('"', '')#re.sub('"', '\"', narrator.h1.text.strip())
            result['Bio'] = narrator.p.text
            result['Title'] = ""
            result['Interviewer'] = ""
            result['Location'] = ""
            result['Date'] = ""
            result['Densho ID'] = ""
            result['transcript'] = {}
            print(result)
            with open(r'C:\\Users\\maganti\\Documents\\INFO 5502 Chen Sir Project UNT\\Web Scraping Interviews\\Narrators Json Files-Updatedwithquestions\\{}.json'.format(result['Narrator_Name']), 'w', encoding='utf-8') as f:
                json.dump(result, f, ensure_ascii=False, indent=4)
            #interview_count = interview_count + 1
            print("****************************")
    #break
    
    
    
print("Total count of narrators is: {}".format(count))